In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/command-qa/command_qa.jsonl
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/adapter_model.safetensors
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/training_args.bin
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/adapter_config.json
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/README.md
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/tokenizer.json
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/tokenizer_config.json
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/special_tokens_map.json
/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final/tokenizer.model
/kaggle/input/title-answer/command_qa (1).jsonl


In [15]:
# !pip install -q transformers datasets accelerate peft bitsandbytes

In [16]:
# !pip install bitsandbytes

In [4]:
from datasets import load_dataset

data_path = "/kaggle/input/title-answer/command_qa (1).jsonl"
dataset = load_dataset("json", data_files=data_path, split="train")

print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': 'How do I undo the most recent local commits in Git?', 'output': 'Undo a commit & redo\n\n```\n$ git commit -m "Something terribly misguided" # (0: Your Accident)\n$ git reset HEAD~                              # (1)\n# === If you just want to undo the commit, stop here! ===\n[ edit files as necessary ]                    # (2)\n$ git add .                                    # (3)\n$ git commit -c ORIG_HEAD                      # (4)\n\n```\n\n\ngit reset is the command responsible for the undo. It will undo your last commit while leaving your working tree (the state of your files on disk) untouched. You\'ll need to add them again before you can commit them again.\nMake corrections to working tree files.\ngit add anything that you want to include in your new commit.\nCommit the changes, reusing the old commit message. reset copied the old head to .git/ORIG_HEAD; commit with -c ORIG_HEAD will open an editor, which initially contains the log message from the old commit an

In [5]:
import re
import re

def clean_instruction(example):
    text = example['instruction']
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = "\n".join(line.rstrip() for line in text.splitlines())
    text = re.sub(r'[ \t]{2,}', ' ', text)  # collapse tabs or multiple spaces
    example['instruction'] = text.strip()
    return example


dataset_process = dataset.map(clean_instruction)

print(dataset_process[0])

Map:   0%|          | 0/1429 [00:00<?, ? examples/s]

{'instruction': 'How do I undo the most recent local commits in Git?', 'output': 'Undo a commit & redo\n\n```\n$ git commit -m "Something terribly misguided" # (0: Your Accident)\n$ git reset HEAD~                              # (1)\n# === If you just want to undo the commit, stop here! ===\n[ edit files as necessary ]                    # (2)\n$ git add .                                    # (3)\n$ git commit -c ORIG_HEAD                      # (4)\n\n```\n\n\ngit reset is the command responsible for the undo. It will undo your last commit while leaving your working tree (the state of your files on disk) untouched. You\'ll need to add them again before you can commit them again.\nMake corrections to working tree files.\ngit add anything that you want to include in your new commit.\nCommit the changes, reusing the old commit message. reset copied the old head to .git/ORIG_HEAD; commit with -c ORIG_HEAD will open an editor, which initially contains the log message from the old commit an

In [6]:
# for datast in range(len(dataset)):
#     print(dataset[datast]['instruction'])

total_length = 0
max_length = 0

for item in dataset_process:
    instruction_length = len(item['instruction'])
    total_length += instruction_length
    if instruction_length > max_length:
        max_length = instruction_length

average_length = total_length / len(dataset_process)

print("Max instruction length:", max_length)
print("Average instruction length:", average_length)



Max instruction length: 170
Average instruction length: 52.87263820853744


In [7]:
total_length = 0
max_length = 0

for item in dataset_process:
    output_length = len(item['output'])
    total_length += output_length
    if output_length > max_length:
        max_length = output_length

average_length = total_length / len(dataset_process)

print("Max output length:", max_length)
print("Average output length:", average_length)

Max output length: 15203
Average output length: 896.0468859342197


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    # Format the full text
    text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    
    # Tokenize with proper formatting
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=2048,
        padding="max_length",  # Ensures fixed-length sequences
        return_tensors="pt"    # Returns PyTorch tensors
    )
    
    # Create labels (mask instruction tokens)
    instruction_text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    instruction_tokens = tokenizer(
        instruction_text,
        truncation=True,
        max_length=150,
        add_special_tokens=False
    )
    
    # Create labels (-100 for instruction, token IDs for response)
    labels = [-100] * len(instruction_tokens["input_ids"]) + tokenized["input_ids"][0][len(instruction_tokens["input_ids"]):].tolist()
    
    return {
        "input_ids": tokenized["input_ids"][0].tolist(),  # Convert to list
        "attention_mask": tokenized["attention_mask"][0].tolist(),
        "labels": labels
    }


# Process dataset with error handling
try:
    tokenized_dataset = dataset_process.map(
        tokenize,
        batched=False,  # Process one example at a time for stability
        remove_columns=["instruction", "output"]
    )
except Exception as e:
    print(f"Error during tokenization: {e}")
    # Add debug to find problematic examples
    for i, example in enumerate(dataset):
        try:
            tokenize(example)
        except Exception as ex:
            print(f"Error in example {i}: {ex}")
            print(example)
            break

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/1429 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

In [11]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1286
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 143
    })
})

In [ ]:
tokenized_dataset['train'][10]

In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found 

In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

base_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)


2025-06-18 17:04:46.562368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750266286.762398      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750266286.815177      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [13]:
sample = tokenized_dataset["train"][0]
print(f"Input IDs shape: {len(sample['input_ids'])}")
print(f"Attention mask shape: {len(sample['attention_mask'])}")
print(f"Labels shape: {len(sample['labels'])}")

Input IDs shape: 2048
Attention mask shape: 2048
Labels shape: 2048


In [18]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

# import torch
# torch.cuda.set_device(device)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./qlora_tinyllama_cli",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # label_names=["input_ids", "attention_mask", "labels"]
)


trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
0,1.825600,1.869084


TrainOutput(global_step=80, training_loss=1.872579574584961, metrics={'train_runtime': 3963.2701, 'train_samples_per_second': 0.324, 'train_steps_per_second': 0.02, 'total_flos': 1.634233982386176e+16, 'train_loss': 1.872579574584961, 'epoch': 0.995334370139969})

In [19]:
!nvidia-smi

Wed Jun 18 18:12:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             35W /  250W |    4673MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [23]:
import evaluate
trainer.evaluate()

{'eval_loss': 1.8690844774246216,
 'eval_runtime': 114.4551,
 'eval_samples_per_second': 1.249,
 'eval_steps_per_second': 0.629,
 'epoch': 0.995334370139969}

In [ ]:
# Move model/data to GPU 1
# model = model.to(1)
# input_tensor = input_tensor.to(device)

In [ ]:
# import torch
# from IPython.display import clear_output

# # Clear GPU cache
# torch.cuda.empty_cache()

# # Clear notebook output (optional)
# clear_output()

# # Verify memory is freed
# print(f"Memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
# print(f"Memory reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

In [21]:
trainer.save_model("./qlora_tinyllama_cli_final")
tokenizer.save_pretrained("./qlora_tinyllama_cli_final")

('./qlora_tinyllama_cli_final/tokenizer_config.json',
 './qlora_tinyllama_cli_final/special_tokens_map.json',
 './qlora_tinyllama_cli_final/tokenizer.model',
 './qlora_tinyllama_cli_final/added_tokens.json',
 './qlora_tinyllama_cli_final/tokenizer.json')

In [24]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./qlora_tinyllama_cli_final", tokenizer=tokenizer, device=0)
prompt = (
    "### Instruction:\n"
    "How can I delete a remote Git branch?\n\n"
    "### Response:\n"
)

output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)[0]["generated_text"]
print(output)

Device set to use cuda:0


### Instruction:
How can I delete a remote Git branch?

### Response:
You can do this using Git's remote delete command:

```
git push --delete remote/branchname

```

This will delete the branch from the remote and remove it from your local repository.
That's it.

Note that if a branch has been merged into master or origin/master, it won't be deleted. In these cases, you will need to do a new push to delete the branch.

```
git push origin --delete branchname

```

If you still need to delete a remote branch, you can use the delete-remote command:

```
git remote delete remote/branchname

```

This will remove the remote and the branch from the remote repository.
Note that you can't delete a remote branch that is referenced by a local branch.

A couple of side notes:

* Do not use --delete-branch when deleting a branch that has been merged into a remote repository. This is because Git does not remove the remote branch.

* Be careful when using the --delete-branch option when deleting 

In [27]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./qlora_tinyllama_cli_final", tokenizer=tokenizer, device=0)
prompt = (
    "### Instruction:\n"
    "How can I delete a remote Git branch?\n\n"
    "### Response:\n"
)

output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)[0]["generated_text"]
print(output)

Device set to use cuda:0


### Instruction:
How can I delete a remote Git branch?

### Response:
You can delete a remote Git branch with the following command:

git push origin --delete BRANCH_NAME

This will delete the branch with the same name as the remote branch, which you need to have created locally before.

If you want to delete the branch from the remote repository without deleting it locally, you can use the following command:

git push origin --delete BRANCH_NAME:BRANCH_NAME

Note that the --delete option is not necessary for all remote branches (the default behavior is not to delete remote branches). You can also use the --force option to delete branches that you don't want to delete, and you can use the --force-with-lease option to delete branches that are currently being merged.

To delete any local branch that is not remotely related to a remote branch, you can use the following command:

git branch -m BRANCH_NAME

This will remove any local branch that is not remotely related to a remote branch, w

In [28]:
!zip -r /kaggle/working/qlora_tinyllama_cli_final.zip /kaggle/working/qlora_tinyllama_cli_final

  adding: kaggle/working/qlora_tinyllama_cli_final/ (stored 0%)
  adding: kaggle/working/qlora_tinyllama_cli_final/adapter_config.json (deflated 53%)
  adding: kaggle/working/qlora_tinyllama_cli_final/tokenizer.json (deflated 85%)
  adding: kaggle/working/qlora_tinyllama_cli_final/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: kaggle/working/qlora_tinyllama_cli_final/tokenizer_config.json (deflated 68%)
  adding: kaggle/working/qlora_tinyllama_cli_final/README.md (deflated 66%)
  adding: kaggle/working/qlora_tinyllama_cli_final/training_args.bin (deflated 52%)
  adding: kaggle/working/qlora_tinyllama_cli_final/tokenizer.model (deflated 55%)
  adding: kaggle/working/qlora_tinyllama_cli_final/special_tokens_map.json (deflated 73%)


In [29]:
eval_prompts = [
    "Create a new Git branch and switch to it.",
    "Compress the folder reports into reports.tar.gz.",
    "List all Python files in the current directory recursively.",
    "Set up a virtual environment and install requests.",
    "Fetch only the first ten lines of a file named output.log.",
    "Clone a repository using git command.",  # Edge case 1
    "How can I delete a remote Git branch?",        # Edge case 2
]


In [30]:
# pip install -U bitsandbytes

In [40]:
pipe = pipeline("text-generation", model="./qlora_tinyllama_cli_final", tokenizer=tokenizer, device=0)
prompt = (
    "### Instruction:\n"
    "How to push in to git?\n\n"
    "### Response:\n"
)

output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.3)[0]["generated_text"]
print(output)

Device set to use cuda:0


### Instruction:
How to push in to git?

### Response:
You can use the following command to push to the remote repository:

```
git push origin master

```

This will push the changes from your local repository to the remote repository.

If you want to push to a specific branch, you can use the following command:

```
git push origin branch_name

```

This will push the changes to the branch named branch_name.

If you want to push to a specific commit, you can use the following command:

```
git push origin HEAD:commit_id

```

This will push the changes to the commit with the given commit ID.

If you want to push to a specific commit, you can use the following command:

```
git push origin commit_id:commit_id

```

This will push the changes to the commit with the given commit ID.

If you want to push to a specific branch and commit, you can use the following command:

```
git push origin branch_name:commit_id

```

This will push the changes to the branch named branch_name and the co

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch

# Load base model
base_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "/kaggle/input/model/transformers/default/1/kaggle/working/qlora_tinyllama_cli_final"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load and merge PEFT adapter
peft_model = PeftModel.from_pretrained(base_model, adapter_path)
# merged_model = peft_model.merge_and_unload()

# Build pipelines
pipe_base = pipeline("text-generation", model=base_model, tokenizer=tokenizer)



In [42]:
pipe = pipeline("text-generation", model="./qlora_tinyllama_cli_final", tokenizer=tokenizer, device=0)
# prompt = (
#     "### Instruction:\n"
#     "How to push in to git?\n\n"
#     "### Response:\n"
# )

# output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.3)[0]["generated_text"]

Device set to use cuda:0


In [32]:
def get_outputs(pipe, prompts):
    outputs = []
    for prompt in prompts:
        full_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"
        result = pipe(full_prompt, max_new_tokens=200, do_sample=False)[0]['generated_text']
        response = result.split("### Response:")[-1].strip()
        outputs.append(response)
    return outputs

base_outputs = get_outputs(pipe_base, eval_prompts)
# finetuned_outputs = get_outputs(pipe_finetuned, eval_prompts)


In [44]:
finetuned_outputs=get_outputs(pipe, eval_prompts)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [45]:
finetuned_outputs

['You can create a new branch by typing git branch -m branchname.\n\nThen you can switch to it by typing git checkout branchname.\n\nYou can also switch back to the previous branch by typing git checkout -b branchname.\n\nYou can also switch to a branch by typing git checkout branchname.\n\nYou can also switch back to the previous branch by typing git checkout -b branchname.\n\nYou can also switch to a branch by typing git checkout branchname.\n\nYou can also switch back to the previous branch by typing git checkout -b branchname.\n\nYou can also switch to a branch by typing git checkout branchname.\n\nYou can also switch back to the previous branch by typing git checkout -b branchname.\n\nYou can also switch to a branch by typing git checkout branchname.\n\nYou can also switch back to the previous branch by typing git checkout -b branchname.\n\nYou can also switch',
 'You can use tar command to compress the folder reports into reports.tar.gz.\n\n```\ntar -czvf reports.tar.gz reports\n

In [46]:
len(finetuned_outputs)

7

In [33]:
base_outputs

['git checkout -b branch_name\n\nThis will create a new branch called branch_name.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout -b branch_name will create a new branch called branch_name.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.\n\ngit checkout branch_name will switch to the branch you just created.',
 'You can use tar command to compress the folder reports into reports.tar.gz.\n\n```\ntar -czvf reports.tar.gz reports\

In [54]:
for i in range(len(base_outputs)):
    print("_______________________________________")
    print(f"{i+1}:𝓕𝓲𝓷𝓮 𝓣𝓾𝓷𝓮 𝓶𝓸𝓭𝓮𝓵 𝓞𝓾𝓽𝓾𝓹𝓽: {finetuned_outputs[i]}, \n {i+1}:𝓑𝓪𝓼𝓮 𝓶𝓸𝓭𝓮𝓵 𝓞𝓾𝓽𝓾𝓹𝓽: {base_outputs[i]}")
    print("_______________________________________")

_______________________________________
1:𝓕𝓲𝓷𝓮 𝓣𝓾𝓷𝓮 𝓶𝓸𝓭𝓮𝓵 𝓞𝓾𝓽𝓾𝓹𝓽: You can create a new branch by typing git branch -m branchname.

Then you can switch to it by typing git checkout branchname.

You can also switch back to the previous branch by typing git checkout -b branchname.

You can also switch to a branch by typing git checkout branchname.

You can also switch back to the previous branch by typing git checkout -b branchname.

You can also switch to a branch by typing git checkout branchname.

You can also switch back to the previous branch by typing git checkout -b branchname.

You can also switch to a branch by typing git checkout branchname.

You can also switch back to the previous branch by typing git checkout -b branchname.

You can also switch to a branch by typing git checkout branchname.

You can also switch back to the previous branch by typing git checkout -b branchname.

You can also switch, 
 1:𝓑𝓪𝓼𝓮 𝓶𝓸𝓭𝓮𝓵 𝓞𝓾𝓽𝓾𝓹𝓽: git checkout -b branch_name

This will create a new bran

In [56]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d3f65db6313320dac9e6290164344d9e4839953c3aa1fd520466096cb23e50b7
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [70]:
import evaluate

# Initialize metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

references = [
    "git checkout -b branch_name",
    "tar -czvf reports.tar.gz reports",
    "import os\nfor root, dirs, files in os.walk(path): print(file)",
    "pip install requests",
    "head -n 10 output.log",
    "git clone https://github.com/username/repo.git",
    "git push origin :branch_name"
]


bleu_base = bleu.compute(
    predictions=base_outputs,
    references=[[ref] for ref in references]
)

bleu_finetuned = bleu.compute(
    predictions=finetuned_outputs,
    references=[[ref] for ref in references]
)

# ROUGE expects plain string lists
rouge_base = rouge.compute(
    predictions=base_outputs,
    references=references,
    rouge_types=["rougeL"]
)

rouge_finetuned = rouge.compute(
    predictions=finetuned_outputs,
    references=references,
    rouge_types=["rougeL"]
)

# Print results
print("📏 BLEU Scores:")
print("Base:      ", bleu_base["bleu"])
print("Fine-tuned:", bleu_finetuned["bleu"])

print("\n📏 ROUGE-L Scores:")
print("Base:      ", rouge_base["rougeL"])
print("Fine-tuned:", rouge_finetuned["rougeL"])


📏 BLEU Scores:
Base:       0.39041142250733285
Fine-tuned: 0.3507226337696886

📏 ROUGE-L Scores:
Base:       0.9427639370845717
Fine-tuned: 0.8266996616935094
